## RESNET 32

 - Training Dataset:  CutMix, beta=1, cutmix_prob=0.5
 - Sagemaker Notebook must be of type, conda_pytorch_p36
 
#### Install Requirements

In [1]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

In [2]:
!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

# Need to add this to requirements.txt
!pip install tensorboard

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


#### Train the Model

In [5]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/resnet.yaml \
    model.resnet.depth 32 \
    train.batch_size 128 \
    dataset.name CIFAR10_CM_.5 \
    train.base_lr 0.1 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_cm_1_.25/exp00 \
    scheduler.epochs 400

[2020-07-11 22:44:39] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_CM_1
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 32
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottleneck
    

#### Refine the Model

In [8]:
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config /home/ec2-user/SageMaker/experiments/resnet_basic_32_cm_1_.25/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_cm_1_.25/exp00/checkpoint_00400.pth \
    dataset.name CIFAR10 \
    train.base_lr .001 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_cm_1_.25/exp00_resume400_50 \
    scheduler.epochs 50

#### Set LEARNING RATE based on ending LR

[2020-07-12 03:34:47] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 32
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_typ

In [55]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_cm_1_.25/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_cm_1_.25/exp00_resume400_50/test_results_0050_cifar10

[2020-07-10 14:10:31] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_cm_1_1/exp00_resume400_50/checkpoint_00050.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:02<00:00, 26.66it/s]
[2020-07-10 14:10:35] __main__ INFO: Elapsed 2.97
[2020-07-10 14:10:35] __main__ INFO: Loss 0.4049 Accuracy 0.9010


In [56]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_cm_1_.25/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_cm_1_.25/exp00_resume400_50/test_results_0050_cifar101

[2020-07-10 14:10:51] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_cm_1_1/exp00_resume400_50/checkpoint_00050.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 17.67it/s]
[2020-07-10 14:10:52] __main__ INFO: Elapsed 0.91
[2020-07-10 14:10:52] __main__ INFO: Loss 0.8114 Accuracy 0.8110


In [57]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_cm_1_.25/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_cm_1_.25/exp00/test_results_0400_cifar10

[2020-07-10 14:11:23] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_cm_1_1/exp00/checkpoint_00400.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:02<00:00, 27.59it/s]
[2020-07-10 14:11:27] __main__ INFO: Elapsed 2.87
[2020-07-10 14:11:27] __main__ INFO: Loss 0.5840 Accuracy 0.8608


In [58]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_cm_1_.25/exp00/checkpoint_00400.pth \
   dataset.name CIFAR101 \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_cm_1_.25/exp00/test_results_0400_cifar101

[2020-07-10 14:11:36] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_cm_1_1/exp00/checkpoint_00400.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 17.42it/s]
[2020-07-10 14:11:37] __main__ INFO: Elapsed 0.92
[2020-07-10 14:11:37] __main__ INFO: Loss 1.0267 Accuracy 0.7240


#### Record the Results

In [3]:
# Write the results to a CSV file so that we can analyze later.
import pandas as pd

results = {'Model': ['resnet_basic_32', 'resnet_basic_32', 'resnet_basic_32']
           'Testset': ['cifar10', 'cifar10', 'cifar10.1']
           'Loss': [0.3604, 0.4011, 0.8051],
           'Epoch': [100, 160, 160],
           'Accuracy': [0.9170, 0.9232, 0.8320],
           'Original_Accuracy': [92.5, 92.5, 84.9],
           'Original_CI': [(92.0, 93.0), (92.0, 93.0), (83.2, 86.4)]
           }

df = pd.DataFrame(results, columns = ['Model', 'Testset', 'Epoch', 'Loss', 'Accuracy', 
                                      'Original_Accuracy', 'Original_CI'])


df.to_csv('/home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/results.csv')
df.head()

,Model,Testset,Epoch,Loss,Accuracy,Original_Accuracy,Original_CI
0,resnet_basic_32,cifar10,100,0.3604,0.9170,92.5,"(92.0, 93.0)"
1,resnet_basic_32,cifar10,160,0.4011,0.9232,92.5,"(92.0, 93.0)"
2,resnet_basic_32,cifar10.1,160,0.8051,0.8320,84.9,"(83.2, 86.4)"


In [61]:
import pandas as pd
model = 'resnet_basic_32_cm_1_.5'

a = pd.Series([model, 400, 'cifar10' ])
c = pd.Series([model, 400, 'cifar10.1' ])

e = pd.Series([model, 50, 'cifar10.1' ])
f = pd.Series([model, 50, 'cifar10' ])
               
df_results = pd.concat([a,c,e,f], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy'] = df_results.apply((lambda row: 92.5 if row[2] == 'cifar10' else 84.9), axis=1)
df_results['Original_CI'] = df_results.apply((lambda row: (92.0, 93.0) if row[2] == 'cifar10' else (83.2, 86.4)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/' + model + '/results.csv')
df_results

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI
0,resnet_basic_32_cm_1_1,400,cifar10,0.584,0.8608,92.5,"(92.0, 93.0)"
1,resnet_basic_32_cm_1_1,400,cifar10.1,1.0267,0.724,84.9,"(83.2, 86.4)"
2,resnet_basic_32_cm_1_1,50,cifar10.1,0.8114,0.811,84.9,"(83.2, 86.4)"
3,resnet_basic_32_cm_1_1,50,cifar10,0.4049,0.901,92.5,"(92.0, 93.0)"


In [118]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-june29'
prefix = 'sagemaker/results/original-models/resnet_basic_32_cm_1_.25'
path = '/home/ec2-user/SageMaker/experiments/resnet_basic_32_cm_1_.25'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)